In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\train_processed.csv")
df.head()


,sample_id,catalog_content,image_link,price,item_name,bullet_points,product_description,value,unit,quantity_standard,unit_standard,quantity_numeric,quantity_count,price_per_unit
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",NaN,NaN,72.00,Fl,NaN,Fl_Oz,2129.292000,NaN,0.002297
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",Original Butter Cookies: Classic butter cookie...,NaN,32.00,Ounce,907.184000,Ounce,907.184000,NaN,0.014462
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",Loaded with hearty long grain wild rice and ve...,NaN,11.40,Ounce,323.184300,Ounce,323.184300,NaN,0.006096
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,"Add to your favorite appetizers, dips & spread...",Judees Powdered Blue Cheese cheddar cheese pow...,11.25,Ounce,318.931875,Ounce,318.931875,NaN,0.095130
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...","kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",NaN,12.00,Count,12.000000,Count,12.000000,12.0,NaN


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re

# Combine text features
df["text_all"] = (
    df["item_name"].fillna("") + " " +
    df["bullet_points"].fillna("") + " " +
    df["product_description"].fillna("")
)

# Basic text cleaning
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9 ]', ' ', text)  # remove special chars
    text = re.sub(r'\s+', ' ', text)             # normalize spaces
    return text.lower().strip()

df["text_all"] = df["text_all"].apply(clean_text)

In [9]:
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)  # include unigrams and bigrams
)

X_tfidf = tfidf.fit_transform(df["text_all"])
print("TF-IDF shape:", X_tfidf.shape)

TF-IDF shape: (75000, 5000)


In [10]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

# Select numeric features
numeric_features = ["quantity_numeric", "price_per_unit", "value"]
X_num = df[numeric_features].fillna(0)

# One-hot encode unit_standard
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
X_cat = ohe.fit_transform(df[["unit_standard"]])

# Combine all (TF-IDF + numeric + categorical)
from scipy.sparse import csr_matrix
X_final = hstack([csr_matrix(X_num.values), X_cat, X_tfidf])

y = df["price"]

print("Final feature shape:", X_final.shape)

Final feature shape: (75000, 5016)


In [11]:
import numpy as np

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0  # avoid division by zero
    return np.mean(diff) * 100


In [25]:
X_train_split, X_val_split, y_train_split, y_val = train_test_split(
    X_train_final, y_train, test_size=0.2, random_state=42
)


In [26]:
from scipy.sparse import hstack, csr_matrix
from catboost import CatBoostRegressor

In [27]:
numeric_features = ["quantity_numeric", "price_per_unit", "value"]
for col in numeric_features:
    if col not in df.columns:
        df[col] = 0
X_num = df[numeric_features].fillna(0)

In [28]:
df['unit_standard'] = df['unit_standard'].fillna('MISSING')

In [29]:
text_features_train = tfidf.transform(df['text_all'])

In [30]:
X_train_final = hstack([csr_matrix(X_num.values), text_features_train])
y_train = np.log1p(df['price'].values) 

In [31]:
cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSE',
    verbose=100
)

In [32]:
cat_model.fit(X_train_final, y_train)

0:	learn: 0.9014194	total: 620ms	remaining: 10m 19s
100:	learn: 0.4089185	total: 53.7s	remaining: 7m 57s
200:	learn: 0.3803679	total: 1m 42s	remaining: 6m 45s
300:	learn: 0.3655460	total: 2m 29s	remaining: 5m 47s
400:	learn: 0.3547536	total: 3m 16s	remaining: 4m 54s
500:	learn: 0.3460497	total: 4m 7s	remaining: 4m 6s
600:	learn: 0.3389694	total: 4m 58s	remaining: 3m 18s
700:	learn: 0.3327708	total: 5m 45s	remaining: 2m 27s
800:	learn: 0.3273778	total: 6m 30s	remaining: 1m 37s
900:	learn: 0.3221159	total: 7m 17s	remaining: 48.1s
999:	learn: 0.3174005	total: 8m 3s	remaining: 0us


In [33]:
y_val_pred_log = cat_model.predict(X_val_split)
y_val_pred = np.expm1(y_val_pred_log) 

In [34]:
rmse = np.sqrt(mean_squared_error(np.expm1(y_val), y_val_pred))
print("✅ RMSE:", rmse)

✅ RMSE: 23.593199115258138


In [35]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0  # avoid division by zero
    return np.mean(diff) * 100

smape_val = smape(np.expm1(y_val), y_val_pred)
print("✅ SMAPE:", smape_val)

✅ SMAPE: 17.327572641675907
